# Préparation pour le Machine Learning

In [43]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

df_movies = pd.read_csv("../01_Data_cleaned/movies.csv", sep=',', index_col=0)
df_movies = df_movies.drop(columns=["level_0", "index"])
df_movies = df_movies[df_movies['titleType'] == "movie"]
df_movies.head(3)

,tconst,titleType,originalTitle,isAdult,startYear,runtimeMinutes,genres,directors,writers,averageRating,...,spoken_languages,vote_average,vote_count,actors_name,actors_rank,directors_name,writers_name,producers_name,cinematographers_name,editors_name
91,tt0000574,movie,The Story of the Kelly Gang,False,1906,70.0,"Action,Adventure,Biography",nm0846879,nm0846879,6.0,...,['xx'],5.375,28,"['Elizabeth Tait', 'Bella Cola', 'Charles Tait']","[1, 6, 11]",['Charles Tait'],['Charles Tait'],"['W.A. Gibson', 'Millard Johnson', 'John Tait'...","['Millard Johnson', 'Orrie Perry', 'Reg Perry']",Unknown
97,tt0000591,movie,L'enfant prodigue,False,1907,90.0,Drama,nm0141150,nm0141150,5.7,...,['xx'],0.000,0,"['Christiane Mandelys', 'Gilberte Sergy', 'Mic...","[3, 4, 5]",['Michel Carré'],['Michel Carré'],Unknown,Unknown,Unknown
241,tt0001184,movie,Don Juan de Serrallonga,False,1910,58.0,"Adventure,Drama","nm0063413,nm0550220",nm0049370,3.8,...,['xx'],1.000,1,"['Dolores Puchol', 'Ricardo de Baños', 'Albert...","[1, 3, 4]","['Ricardo de Baños', 'Alberto Marro']",['Víctor Balaguer'],Unknown,['Ramón de Baños'],Unknown


In [44]:
df_movies['titleType'].value_counts()

titleType
movie    191270
Name: count, dtype: int64

In [45]:
# informations générales de la base
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 191270 entries, 91 to 235135
Data columns (total 30 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   tconst                 191270 non-null  object 
 1   titleType              191270 non-null  object 
 2   originalTitle          191270 non-null  object 
 3   isAdult                191270 non-null  bool   
 4   startYear              191270 non-null  int64  
 5   runtimeMinutes         191270 non-null  float64
 6   genres                 191270 non-null  object 
 7   directors              191270 non-null  object 
 8   writers                191270 non-null  object 
 9   averageRating          191270 non-null  float64
 10  numVotes               191270 non-null  float64
 11  periode                191270 non-null  int64  
 12  budget                 191270 non-null  int64  
 13  id                     191270 non-null  int64  
 14  original_language      191270 non-null  

In [46]:
# check valeurs manquantes
df_movies.isna().sum()

tconst                   0
titleType                0
originalTitle            0
isAdult                  0
startYear                0
runtimeMinutes           0
genres                   0
directors                0
writers                  0
averageRating            0
numVotes                 0
periode                  0
budget                   0
id                       0
original_language        0
overview                 0
popularity               0
poster_path              0
production_countries     0
revenue                  0
spoken_languages         0
vote_average             0
vote_count               0
actors_name              0
actors_rank              0
directors_name           0
writers_name             0
producers_name           0
cinematographers_name    0
editors_name             0
dtype: int64

In [47]:
# descriptif statistiques
df_movies.describe()

,startYear,runtimeMinutes,averageRating,numVotes,periode,budget,id,popularity,revenue,vote_average,vote_count
count,191270.000000,191270.000000,191270.000000,1.912700e+05,191270.000000,1.912700e+05,1.912700e+05,191270.000000,1.912700e+05,191270.000000,191270.000000
mean,1994.195399,94.972400,6.022176,5.985674e+03,1989.746798,1.379140e+06,3.549511e+05,3.511111,3.615819e+06,4.501696,100.637800
std,26.603984,21.916737,1.218017,4.692797e+04,26.836747,1.034081e+07,2.978892e+05,26.745629,3.832356e+07,2.743128,744.444926
min,1903.000000,1.000000,1.000000,5.000000e+00,1910.000000,0.000000e+00,2.000000e+00,0.600000,0.000000e+00,0.000000,0.000000
25%,1977.000000,83.000000,5.300000,4.600000e+01,1970.000000,0.000000e+00,9.832875e+04,0.731000,0.000000e+00,2.400000,1.000000
50%,2005.000000,92.000000,6.200000,1.770000e+02,2000.000000,0.000000e+00,2.805165e+05,1.365000,0.000000e+00,5.400000,4.000000
75%,2016.000000,104.000000,6.800000,7.930000e+02,2010.000000,0.000000e+00,5.364738e+05,2.655000,0.000000e+00,6.500000,14.000000
max,2025.000000,240.000000,10.000000,2.959184e+06,2020.000000,5.793304e+08,1.122413e+06,5089.969000,2.923706e+09,10.000000,33630.000000


## Transformation de la colonne "genres" : extraire les genres uniques et les transformer en colonnes booléenes

#### Fonction transformant la chaine de fils en liste et création de la colonne genres_liste

In [48]:
def split_chaine_en_liste(x):
    if isinstance(x, str): # si x est une chaine de caractère, 
        return x.split(',') # applique le split pour transformer la chaine en liste de plusieurs chaines de caractères
    else:
        return x # sinon retourne x tel quel


In [49]:
df_movies['genres_liste'] = df_movies['genres'].apply(split_chaine_en_liste)
df_movies['genres_liste']

91        [Action, Adventure, Biography]
97                               [Drama]
241                   [Adventure, Drama]
323                            [Unknown]
346                              [Drama]
                       ...              
235131                          [Family]
235132                     [Documentary]
235133     [Action, Adventure, Thriller]
235134                        [Thriller]
235135                  [Drama, History]
Name: genres_liste, Length: 191270, dtype: object

#### Extraction des genres uniques dans toutes les listes

In [50]:
# extraire les genres uniques 
tous_les_genres = set()
for genres in df_movies['genres_liste']:
  tous_les_genres.update(genres)

tous_les_genres

{'Action',
 'Adult',
 'Adventure',
 'Animation',
 'Biography',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'Film-Noir',
 'Game-Show',
 'History',
 'Horror',
 'Music',
 'Musical',
 'Mystery',
 'News',
 'Reality-TV',
 'Romance',
 'Sci-Fi',
 'Sport',
 'Talk-Show',
 'Thriller',
 'Unknown',
 'War',
 'Western'}

#### Création d'autant de colonnes binaires que de genres uniques

In [51]:
import warnings
warnings.filterwarnings("ignore") 

# créer des colonnes binaires pour chaque genre unique
for genre in tous_les_genres:
    def genre_present(x):
        return int(genre in x)
    
    df_movies[f'genre_{genre}'] = df_movies['genres_liste'].apply(genre_present)

df_movies.columns

Index(['tconst', 'titleType', 'originalTitle', 'isAdult', 'startYear',
       'runtimeMinutes', 'genres', 'directors', 'writers', 'averageRating',
       'numVotes', 'periode', 'budget', 'id', 'original_language', 'overview',
       'popularity', 'poster_path', 'production_countries', 'revenue',
       'spoken_languages', 'vote_average', 'vote_count', 'actors_name',
       'actors_rank', 'directors_name', 'writers_name', 'producers_name',
       'cinematographers_name', 'editors_name', 'genres_liste',
       'genre_Western', 'genre_Thriller', 'genre_Documentary', 'genre_Fantasy',
       'genre_Romance', 'genre_Film-Noir', 'genre_Musical', 'genre_Talk-Show',
       'genre_History', 'genre_Adventure', 'genre_Biography', 'genre_Comedy',
       'genre_Crime', 'genre_Music', 'genre_Sport', 'genre_Unknown',
       'genre_Action', 'genre_Drama', 'genre_Adult', 'genre_Mystery',
       'genre_Family', 'genre_War', 'genre_News', 'genre_Game-Show',
       'genre_Reality-TV', 'genre_Sci-Fi', 'genr

In [52]:
df_movies.sample(5)

,tconst,titleType,originalTitle,isAdult,startYear,runtimeMinutes,genres,directors,writers,averageRating,...,genre_Adult,genre_Mystery,genre_Family,genre_War,genre_News,genre_Game-Show,genre_Reality-TV,genre_Sci-Fi,genre_Animation,genre_Horror
71444,tt0183304,movie,Keung gaan on chung on,False,1993,103.0,Crime,nm1300703,nm1299835,3.9,...,0,0,0,0,0,0,0,0,0,0
68584,tt0167262,movie,Lost in the Legion,False,1934,65.0,Unknown,nm0628345,"nm0141351,nm0183821,nm0367615",5.8,...,0,0,0,0,0,0,0,0,0,0
226409,tt8185374,movie,Staff Only,False,2019,82.0,Drama,nm2843690,"nm2843690,nm2972241",6.0,...,0,0,0,0,0,0,0,0,0,0
204426,tt5003186,movie,Plan de fuga,False,2016,105.0,"Crime,Drama,Thriller",nm1981567,nm1981567,5.6,...,0,0,0,0,0,0,0,0,0,0
50372,tt0099462,movie,Dream Machine,False,1991,88.0,"Action,Comedy,Crime",nm0206767,nm0376301,5.1,...,0,0,0,0,0,0,0,0,0,0


# Base données pour ML

In [54]:
df_movies_new = df_movies[['isAdult', 'runtimeMinutes', 'averageRating', 'numVotes', 'periode', 'budget', 'overview', 'popularity', 
                           'genre_Film-Noir', 'genre_Sci-Fi', 'genre_History', 'genre_Reality-TV', 'genre_News', 'genre_Comedy', 
                           'genre_Mystery', 'genre_Fantasy', 'genre_Family', 'genre_Crime', 'genre_Talk-Show',
                            'genre_Sport', 'genre_Animation', 'genre_Biography', 'genre_Adventure', 'genre_Unknown', 'genre_Game-Show', 
                            'genre_Adult', 'genre_Music', 'genre_War', 'genre_Romance', 'genre_Action', 'genre_Western', 'genre_Horror', 
                            'genre_Documentary', 'genre_Musical', 'genre_Drama', 'genre_Thriller']]

df_movies_new.info()

<class 'pandas.core.frame.DataFrame'>
Index: 191270 entries, 91 to 235135
Data columns (total 36 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   isAdult            191270 non-null  bool   
 1   runtimeMinutes     191270 non-null  float64
 2   averageRating      191270 non-null  float64
 3   numVotes           191270 non-null  float64
 4   periode            191270 non-null  int64  
 5   budget             191270 non-null  int64  
 6   overview           191270 non-null  object 
 7   popularity         191270 non-null  float64
 8   genre_Film-Noir    191270 non-null  int64  
 9   genre_Sci-Fi       191270 non-null  int64  
 10  genre_History      191270 non-null  int64  
 11  genre_Reality-TV   191270 non-null  int64  
 12  genre_News         191270 non-null  int64  
 13  genre_Comedy       191270 non-null  int64  
 14  genre_Mystery      191270 non-null  int64  
 15  genre_Fantasy      191270 non-null  int64  
 16  genre_

#### Encodage des colonnes pour entrainement

In [55]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)  # limite à 5000 mots pour éviter trop de dimensions
tfidf_matrix = tfidf_vectorizer.fit_transform(df_movies_new['overview'])
tfidf_matrix.shape


(191270, 5000)

In [56]:
from scipy.sparse import hstack

features = ['runtimeMinutes', 'averageRating', 'numVotes', 'popularity', 'budget'] + \
           [col for col in df_movies_new.columns if col.startswith('genre_')]

# encodage des valeurs numériques
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_movies_new[features])

X_final = hstack([X_scaled, tfidf_matrix]) # fusionner les caractéristiques numériques et la matrice TF-IDF
X_final.shape

(191270, 5033)

# Créer et entraîner le modèle KNN

In [57]:
knn = NearestNeighbors(n_neighbors=10, metric='cosine')  # On utilise la similarité cosinus pour le texte
knn.fit(X_final)

NearestNeighbors(metric='cosine', n_neighbors=10)

# Fonction de recommandation des films

In [89]:
def recommander_films(film_titre, df_movies, df_movies_new, features, scaler, tfidf_vectorizer, knn):
    """
    Trouve des films similaires à celui donné en entrée en utilisant KNN.
    
    - film_titre : titre du film de référence
    - df_movies : DataFrame contenant les informations des films
    - df_movies_new : DataFrame utilisé pour entraîner le modèle KNN
    - features : colonnes utilisées pour KNN
    - scaler : StandardScaler entraîné
    - tfidf_vectorizer : modèle TF-IDF entraîné
    - knn : modèle KNN entraîné
    """
    
    # 📌 Vérifier si le film est bien dans df_movies
    if film_titre not in df_movies['originalTitle'].values:
        return "Film non trouvé dans la base."

    # 📌 Trouver l’index du film dans df_movies
    film_index_movies = df_movies[df_movies['originalTitle'] == film_titre].index[0]

    # 📌 Vérifier que cet index existe dans df_movies_new
    if film_index_movies >= len(df_movies_new):
        return "Correspondance introuvable dans les données d'entraînement."

    # 📌 Extraire ses caractéristiques et standardiser
    film_features = df_movies_new.loc[film_index_movies, features].values.reshape(1, -1)
    film_features_scaled = scaler.transform(film_features)

    # 📌 Transformer `overview` en vecteur TF-IDF
    film_overview = df_movies.loc[film_index_movies, 'overview']
    film_overview_tfidf = tfidf_vectorizer.transform([film_overview])

    # 📌 Fusionner les caractéristiques numériques et le TF-IDF
    film_vector = hstack([film_features_scaled, film_overview_tfidf])

    # 📌 Trouver les films les plus proches
    distances, indices = knn.kneighbors(film_vector)

    # 📌 Récupérer les films recommandés
    recommandations = df_movies.iloc[indices[0][1:]]  # Exclure le film lui-même

    # 📌 Retourner les films avec plus d'infos (titre, période, genres, rating, popularité)
    return recommandations[['originalTitle', 'periode', 'averageRating', 'popularity'] + [col for col in df_movies.columns if col.startswith('genre_')]]


In [90]:
verif = df_movies[df_movies['originalTitle'] == 'AKA']
verif

,tconst,titleType,originalTitle,isAdult,startYear,runtimeMinutes,genres,directors,writers,averageRating,...,genre_Adult,genre_Mystery,genre_Family,genre_War,genre_News,genre_Game-Show,genre_Reality-TV,genre_Sci-Fi,genre_Animation,genre_Horror
91869,tt0317052,movie,AKA,False,2002,123.0,"Drama,Romance",nm0747003,nm0747003,6.3,...,0,0,0,0,0,0,0,0,0,0
182794,tt27197387,movie,AKA,False,2023,122.0,"Action,Crime,Thriller",nm1889990,"nm1889990,nm1839955",6.6,...,0,0,0,0,0,0,0,0,0,0


In [91]:
recommander_films("AKA", df_movies, df_movies_new, features, scaler, tfidf_vectorizer, knn)

,originalTitle,periode,averageRating,popularity,genre_Western,genre_Thriller,genre_Documentary,genre_Fantasy,genre_Romance,genre_Film-Noir,...,genre_Adult,genre_Mystery,genre_Family,genre_War,genre_News,genre_Game-Show,genre_Reality-TV,genre_Sci-Fi,genre_Animation,genre_Horror
24259,Marjorie Morningstar,1950,6.2,1.954,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
92503,Dôtonborigawa,1980,6.4,2.071,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
176856,Eungyo,2010,6.6,12.749,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
116198,"Omoi, Omoware, Furi, Furare",2020,6.3,2.144,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
38220,The Last Tycoon,1970,6.2,8.809,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
212919,Aoharaido,2010,6.1,8.584,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
224002,Satu Hari Nanti,2010,6.4,0.600,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
213988,Sensei!,2010,6.3,5.460,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
183727,Zhi wo men zhong jiang shi qu de qing chun,2010,6.4,5.757,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
